Implementation in Python with numpy of a one vs. all classification method.  Assuming there are N possible classes fit N possible models.  Each of these N models finds likelihood of observation being of class i vs. the probability of being one of the other N - 1 classes.  Afterwards, each observation is assigned to the class which has the maximum probability from the N different models.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import loadmat

In [2]:
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

In [3]:
data = loadmat("ex3data1.mat")
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': 'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
data["X"].shape, data["y"].shape

((5000L, 400L), (5000L, 1L))

In [5]:
def cost(theta, X, y, lamb):
    theta, X, y = np.matrix(theta), np.matrix(X), np.matrix(y) #conversion to numpy matrices if needed
    m = float(len(X))
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = lamb/(2.0 * m) * np.sum(np.power(theta[:, 1:theta.shape[1]], 2)) #regularization term
    return np.sum(first - second) / m + reg

In [6]:
def gradient(theta, X, y, lamb):
    theta, X, y = np.matrix(theta), np.matrix(X), np.matrix(y) #convert to matrices
    error = sigmoid(X * theta.T) - y
    m = float(len(X)) #number of observations
    
    grad = (1.0 / m * X.T * error).T + ((lamb / m)*theta)
    grad[0, 0] = np.sum(np.multiply(error, X[:, 0]))/m #intercept not regularized
    return np.array(grad).ravel()

In [7]:
from scipy.optimize import minimize

In [8]:
def oneVsAll(X, y, num_labels, learn):
    #there are k different possible labels, n parameters, m observations
    rows = X.shape[0] #number of observations
    params = X.shape[1] #number of parameters
    
    #create k X (n + 1) array for parameters of each of k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    #insert column of ones for intercept term
    X = np.insert(X, 0, values = np.ones(rows), axis = 1)
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1) #parameters for label i
        y_i = np.array([1 if label == i else 0 for label in y]) #new y for label i
        y_i = np.reshape(y_i, (rows, 1)) #reshape as m X 1 array
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learn), method="TNC", jac=gradient)
        all_theta[i - 1, :] = fmin.x
        
    return all_theta

In [9]:
#check shape of vectors and matrices
rows = data["X"].shape[0]
params = data["X"].shape[1]
all_theta = np.zeros((10, params + 1))
X = np.insert(data["X"], 0, np.ones(rows), axis=1)
theta = np.zeros(params + 1)
y_0 = np.array([1 if label == 0 else 0 for label in data["y"]])
y_0 = np.reshape(y_0, (rows, 1))
print X.shape, y_0.shape, theta.shape, all_theta.shape

(5000L, 401L) (5000L, 1L) (401L,) (10L, 401L)


In [10]:
np.unique(data["y"])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [11]:
all_theta = oneVsAll(data["X"], data['y'], 10, 1.0)
all_theta

array([[ -2.38269917e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.30407805e-03,  -6.74624648e-10,   0.00000000e+00],
       [ -3.18544074e+00,   0.00000000e+00,   0.00000000e+00, ...,
          4.46013538e-03,  -5.08504735e-04,   0.00000000e+00],
       [ -4.79511162e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.86319768e-05,  -2.46881239e-07,   0.00000000e+00],
       ..., 
       [ -7.98768702e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -8.94530849e-05,   7.21298510e-06,   0.00000000e+00],
       [ -4.57418382e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.33371425e-03,   9.96602811e-05,   0.00000000e+00],
       [ -5.40516157e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.16792563e-04,   7.88480861e-06,   0.00000000e+00]])

In [12]:
print all_theta.shape

(10L, 401L)


In [13]:
def predict(X, all_theta):
    rows = X.shape[0]  #number of observations
    params = X.shape[1] #number of parameters
    num_labels = all_theta.shape[0] #number of class labels
    #insert column of ones
    X = np.insert(X, 0, values = np.ones(rows), axis = 1)
    X, all_theta = np.matrix(X), np.matrix(all_theta) #convert to numpy matrices
    #predictions for all labels for each observation in X
    h = sigmoid(X * all_theta.T)
    h_argmax = np.argmax(h, axis = 1) #return column that maximizes value in each row
    return h_argmax + 1 #adjust for zero index to get true prediction

In [14]:
preds = predict(data["X"], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(preds, data["y"])]
print "Accuracy:", sum(correct)/float(len(correct))

Accuracy: 0.9446


In [16]:
np.unique(np.array(preds))

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [17]:
from collections import Counter

In [31]:
#counts of each label in dataset
counts = dict()
for val[0] in data["y"]:
    if val[0] in counts:
        counts[val[0]] += 1
    else:
        counts[val[0]] = 1
counts

{1: 500,
 2: 500,
 3: 500,
 4: 500,
 5: 500,
 6: 500,
 7: 500,
 8: 500,
 9: 500,
 10: 500}